In [1]:
# HIZON

# Psycopg2 is the most popular PostgreSQL database adapter
# database connector connects the database to any application and the implementation is held through a database adapter
import psycopg2 as pc

# numpy is a package of python that provides extra feature to list
import numpy as np

# creates dataframe that can be visuallized and insert into the database
import pandas as pd

# package that allows python to read csv file type documents
import csv as reader

In [2]:
# HIZON

# where psycopg2 (database adapter) connects with the database (pgadmin4)
conn = pc.connect("dbname = SUSPECTS user = postgres password = @Ngelo8998 host = localhost")
conn
c = conn.cursor()

In [3]:
from sqlalchemy import create_engine

In [4]:
engine = create_engine('postgresql+psycopg2://postgres:@Ngelo8998@localhost:5432/SUSPECTS')

In [6]:
profile_batch_one = pd.read_csv('suspects_data - suspect_profile1_batch1.csv')

In [7]:
profile_batch_two = pd.read_csv('suspects_data - suspect_profile1_batch2.csv')

In [8]:
profile_batch_one.to_sql('suspect_profile1', engine, index = False, if_exists = 'replace')

In [9]:
profile_batch_two.to_sql('suspect_profile1', engine, index = False, if_exists = 'append')

In [11]:
# HIZON

df1 = pd.read_sql_query("SELECT * FROM suspect_profile1", conn)
df1.tail()

,SUSPECT_ID,NAME,SEX,AGE,HEIGHT
495,496,Oren Quinn,1,76,1
496,497,Alvin Park,2,48,2
497,498,Clark Moses,2,43,1
498,499,Graham Cain,1,40,3
499,500,Stacy Bennett,2,49,3


In [12]:
df1.shape

(500, 5)

In [13]:
features_batch_one = pd.read_csv('suspects_data - suspect_features_batch1.csv')

In [14]:
features_batch_two = pd.read_csv('suspects_data - suspect_features_batch2.csv')

In [17]:
features_batch_one.to_sql('suspect_features', engine, index = False, if_exists = 'replace')

In [18]:
features_batch_two.to_sql('suspect_features', engine, index = False, if_exists = 'append')

In [19]:
# HIZON

df2 = pd.read_sql_query("SELECT * FROM suspect_features", conn)
df2.tail()

,ID_NO,HAIR_COLOR,EYE_COLOR,FACIAL_HAIR,TATTOOS,GLASSES,SCARS,FEET_SIZE
495,496,Blonde,Green,0,1,0,1,1
496,497,Red,Blue,0,0,0,0,2
497,498,Brown,Blue,0,1,1,0,3
498,499,Red,Blue,1,0,1,1,3
499,500,Red,Green,0,0,1,0,2


In [20]:
df2.shape

(500, 8)

In [5]:
classification = pd.read_csv('suspects_data - suspect_classification.csv')

#HIZON

In [6]:
classification.to_sql('suspect_classification', engine, index = False, if_exists = 'replace')

In [7]:
df3 = pd.read_sql_query("SELECT * FROM suspect_classification", conn)
df3.tail()

,ID_NO,SUSPECT
495,496,NO
496,497,NO
497,498,YES
498,499,YES
499,500,NO


In [8]:
df3.shape

(500, 2)

In [41]:
#HIZON

query = """
    SELECT 
        suspect_classification."ID_NO", "FACIAL_HAIR", "TATTOOS", "GLASSES", "SCARS", "FEET_SIZE", "SUSPECT",
        COUNT("TATTOOS") as "COUNT_TATTOO", COUNT("SCARS") as "COUNT_SCARS"
    FROM suspect_features 
    INNER JOIN suspect_classification
    ON suspect_features."ID_NO" = suspect_classification."ID_NO"
    WHERE "SUSPECT" LIKE 'YES' 
    GROUP BY suspect_classification."ID_NO", "FACIAL_HAIR", "TATTOOS", "GLASSES", "SCARS", "FEET_SIZE", "SUSPECT"
    HAVING  COUNT("SCARS") = 1 AND COUNT("TATTOOS") = 1
"""

df4 = pd.read_sql_query(query, conn)
df4

,ID_NO,FACIAL_HAIR,TATTOOS,GLASSES,SCARS,FEET_SIZE,SUSPECT,COUNT_TATTOO,COUNT_SCARS
0,1,0,1,1,0,2,YES,1,1
1,2,1,0,0,1,1,YES,1,1
2,3,0,0,1,0,1,YES,1,1
3,7,0,0,0,0,3,YES,1,1
4,8,1,0,1,1,2,YES,1,1
...,...,...,...,...,...,...,...,...,...
220,492,1,1,1,0,3,YES,1,1
221,493,0,1,0,0,3,YES,1,1
222,494,1,1,0,1,2,YES,1,1
223,498,0,1,1,0,3,YES,1,1


In [42]:
df4.shape

(225, 9)

In [5]:
query = """
    SELECT 
        suspect_classification."ID_NO", "FACIAL_HAIR", "TATTOOS", "GLASSES", "SCARS", "FEET_SIZE", "SUSPECT",
        COUNT("TATTOOS") as "COUNT_TATTOO", COUNT("SCARS") as "COUNT_SCARS"
    FROM suspect_features 
    INNER JOIN suspect_classification
    ON suspect_features."ID_NO" = suspect_classification."ID_NO"
    WHERE "SUSPECT" LIKE 'YES' 
    GROUP BY suspect_classification."ID_NO", "FACIAL_HAIR", "TATTOOS", "GLASSES", "SCARS", "FEET_SIZE", "SUSPECT"
    HAVING  COUNT("SCARS") = 0 AND COUNT("TATTOOS") = 0
"""

df5 = pd.read_sql_query(query, conn)
df5

,ID_NO,FACIAL_HAIR,TATTOOS,GLASSES,SCARS,FEET_SIZE,SUSPECT,COUNT_TATTOO,COUNT_SCARS


In [6]:
df5.shape

(0, 9)

In [63]:
# HIZON

query = """
    SELECT "SUSPECT_ID", "NAME", "SEX", "AGE", "HEIGHT", "HAIR_COLOR", "EYE_COLOR", "TATTOOS", "GLASSES", "SCARS", 
        "FACIAL_HAIR","FEET_SIZE", "SUSPECT"
    FROM suspect_profile1 
    INNER JOIN suspect_features
        ON "SUSPECT_ID" = suspect_features."ID_NO"
    INNER JOIN suspect_classification
        ON "SUSPECT_ID" = suspect_classification."ID_NO"
    WHERE "SUSPECT" LIKE 'YES'
"""

df6 = pd.read_sql_query(query, conn)
df6

,SUSPECT_ID,NAME,SEX,AGE,HEIGHT,HAIR_COLOR,EYE_COLOR,TATTOOS,GLASSES,SCARS,FACIAL_HAIR,FEET_SIZE,SUSPECT
0,1,Harold Day,1,28,1,Red,Blue,1,1,0,0,2,YES
1,2,Xaviera Bryan,2,53,2,Black,Green,0,0,1,1,1,YES
2,3,Bill Goodwin,1,32,1,Brown,Blue,0,1,0,0,1,YES
3,7,Xyla Anderson,1,61,2,Blonde,Green,0,0,0,0,3,YES
4,8,Travis Ortiz,1,52,1,Blonde,Brown,0,1,1,1,2,YES
...,...,...,...,...,...,...,...,...,...,...,...,...,...
220,492,Philip Mcknight,1,51,1,Red,Green,1,1,0,1,3,YES
221,493,Lawrie Hammond,2,31,2,Black,Brown,1,0,0,0,3,YES
222,494,Raphael Castaneda,2,73,3,Blonde,Blue,1,0,1,1,2,YES
223,498,Clark Moses,2,43,1,Brown,Blue,1,1,0,0,3,YES


In [10]:
# HIZON

query = """
    SELECT "SUSPECT_ID", "NAME", "SEX", "AGE", "HEIGHT", "HAIR_COLOR", "EYE_COLOR", "TATTOOS", "GLASSES", "SCARS", "SEX"
        "FACIAL_HAIR","FEET_SIZE", "SUSPECT"
    FROM suspect_profile1 
    INNER JOIN suspect_features
        ON "SUSPECT_ID" = suspect_features."ID_NO"
    INNER JOIN suspect_classification
        ON "SUSPECT_ID" = suspect_classification."ID_NO"
    WHERE "SUSPECT" LIKE 'YES' AND "SEX" = 1 AND "SCARS" = 0 AND "TATTOOS" = 0
"""

df6 = pd.read_sql_query(query, conn)
df6.tail()

,SUSPECT_ID,NAME,SEX,AGE,HEIGHT,HAIR_COLOR,EYE_COLOR,TATTOOS,GLASSES,SCARS,FACIAL_HAIR,FEET_SIZE,SUSPECT
21,377,Wyatt Zimmerman,1,35,1,Brown,Brown,0,1,0,1,1,YES
22,408,Quon Buchanan,1,81,2,Blonde,Brown,0,0,0,1,3,YES
23,412,Harl Sanders,1,46,1,Blonde,Brown,0,1,0,1,3,YES
24,420,Lani Foley,1,18,1,Red,Blue,0,0,0,1,3,YES
25,453,Wylie Mendoza,1,65,2,Black,Blue,0,0,0,1,3,YES


In [14]:
# HIZON

query = """
    SELECT * FROM 
    suspect_profile1
    INNER JOIN
    suspect_classification
    ON 
    "SUSPECT_ID" = suspect_classification."ID_NO"
    WHERE "AGE" > 50 AND "SUSPECT" LIKE 'YES'
"""

df7 = pd.read_sql_query(query, conn)
df7.shape

(122, 7)

In [16]:
# HIZON

query = """
    SELECT * FROM 
    suspect_profile1
    INNER JOIN suspect_features
        ON "SUSPECT_ID" = suspect_features."ID_NO"
    INNER JOIN suspect_classification
        ON "SUSPECT_ID" = suspect_classification."ID_NO"
    WHERE "SEX" = 2 AND "SUSPECT" LIKE 'YES' AND "HAIR_COLOR" LIKE 'Blonde'
"""

df8 = pd.read_sql_query(query, conn)
df8.shape

(26, 15)

In [20]:
# HIZON

query = """
    SELECT * FROM 
    suspect_profile1
    INNER JOIN suspect_features
        ON "SUSPECT_ID" = suspect_features."ID_NO"
    INNER JOIN suspect_classification
        ON "SUSPECT_ID" = suspect_classification."ID_NO"
    WHERE "AGE" = 35 AND "SEX" = 1 AND "SUSPECT" LIKE 'YES' AND "EYE_COLOR" LIKE 'Brown' AND "GLASSES" = 1
"""

df9 = pd.read_sql_query(query, conn)
df9

,SUSPECT_ID,NAME,SEX,AGE,HEIGHT,ID_NO,HAIR_COLOR,EYE_COLOR,FACIAL_HAIR,TATTOOS,GLASSES,SCARS,FEET_SIZE,ID_NO,SUSPECT
0,141,Ima Estes,1,35,3,141,Red,Brown,0,1,1,0,1,141,YES
1,188,Nehru Vargas,1,35,1,188,Black,Brown,0,0,1,0,2,188,YES
2,377,Wyatt Zimmerman,1,35,1,377,Brown,Brown,0,0,1,0,1,377,YES


In [25]:
# HIZON

query = """
    SELECT * FROM 
    suspect_profile1
    INNER JOIN suspect_features
        ON "SUSPECT_ID" = suspect_features."ID_NO"
    INNER JOIN suspect_classification
        ON "SUSPECT_ID" = suspect_classification."ID_NO"
    WHERE "AGE" > 60 AND "EYE_COLOR" LIKE 'Green' AND "GLASSES" = 0 AND "SCARS" = 0 AND "SEX" = 2 AND "SUSPECT" LIKE 'YES'
"""

df10 = pd.read_sql_query(query, conn)
df10

,SUSPECT_ID,NAME,SEX,AGE,HEIGHT,ID_NO,HAIR_COLOR,EYE_COLOR,FACIAL_HAIR,TATTOOS,GLASSES,SCARS,FEET_SIZE,ID_NO,SUSPECT
0,118,Tatyana Logan,2,71,2,118,Black,Green,1,1,0,0,3,118,YES
1,316,Quinn Booth,2,78,2,316,Brown,Green,0,1,0,0,1,316,YES
2,342,Faith Beck,2,78,1,342,Brown,Green,0,0,0,0,2,342,YES
3,419,Malike Branch,2,76,3,419,Brown,Green,1,0,0,0,3,419,YES


In [27]:
# HIZON

query = """
    SELECT * FROM 
    suspect_profile1
    INNER JOIN suspect_features
        ON "SUSPECT_ID" = suspect_features."ID_NO"
    INNER JOIN suspect_classification
        ON "SUSPECT_ID" = suspect_classification."ID_NO"
    WHERE "SEX" = 2 AND "SUSPECT" LIKE 'NO' AND "EYE_COLOR" LIKE 'Blue' AND "GLASSES" = 1 AND "SCARS" = 1 AND "FEET_SIZE" = 2
"""

df11 = pd.read_sql_query(query, conn)
df11

,SUSPECT_ID,NAME,SEX,AGE,HEIGHT,ID_NO,HAIR_COLOR,EYE_COLOR,FACIAL_HAIR,TATTOOS,GLASSES,SCARS,FEET_SIZE,ID_NO,SUSPECT
0,11,Madeson Nunez,2,51,1,11,Blonde,Blue,0,1,1,1,2,11,NO
1,70,Britanney Tyson,2,41,3,70,Red,Blue,0,1,1,1,2,70,NO
2,222,Briary Barnett,2,51,1,222,Black,Blue,1,1,1,1,2,222,NO
3,233,Rain Bell,2,52,2,233,Blonde,Blue,0,1,1,1,2,233,NO
4,366,India Bradford,2,73,3,366,Blonde,Blue,0,0,1,1,2,366,NO
5,373,Galena Hester,2,83,3,373,Blonde,Blue,0,1,1,1,2,373,NO
6,491,Lynn Lopez,2,85,1,491,Black,Blue,0,0,1,1,2,491,NO


In [28]:
df11.shape

(7, 15)